# <center>Segmenting and Clustering </center>
## <center>Neighborhoods in Toronto</center>

In [108]:
# import required libraries
import requests
import lxml.html as lh
import pandas as pd

In [109]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
page # request is successful if responce is 200

<Response [200]>

Store the contents of the website under doc and parse data that are stored between tr  of HTML

In [110]:
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

#Check the length of the first 12 rows. All rows should have 3 columns
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

## Parse Table Header
Next, let’s parse the first row as our header.

In [111]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


## Creating Pandas DataFrame
Each header is appended to a tuple along with an empty list.

In [112]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

## Create the DataFrame

In [113]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [114]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


Ignore cells with a borough that is Not assigned.

In [115]:
df = df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n


In [116]:
df.shape

(211, 3)

While scrapping the page, name of columns had \n (new line character) so clean the column names of dataframe

In [117]:
df.columns = ['Postcode', 'Borough', 'Neighbourhood']

Group By columns where Postcode and Borough values are same. Join value of Neighbourhood column with ,

In [118]:
df = df['Neighbourhood'].groupby([df.Postcode, df.Borough]).apply(list).reset_index()

In [119]:
df.shape

(103, 3)

In [120]:
df = df[df.Neighbourhood != 'Not assigned']

In [121]:
df.shape

(103, 3)